In [2]:
import os
import cv2
import albumentations as A
import numpy as np
from glob import glob
from tqdm import tqdm
import shutil

INPUT_DIR = "/kaggle/input/mkr-logos/mkr nns logos.v2i.yolov11/train/images"
OUTPUT_DIR = "/kaggle/working/augmented_dataset"

In [3]:
if os.path.exists(OUTPUT_DIR):
    shutil.rmtree(OUTPUT_DIR)
os.makedirs(f"{OUTPUT_DIR}/images", exist_ok=True)
os.makedirs(f"{OUTPUT_DIR}/labels", exist_ok=True)

transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.Rotate(limit=10, p=0.5),
    A.Perspective(scale=(0.05, 0.1), p=0.3),
    A.RandomScale(scale_limit=0.1, p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.GaussianBlur(blur_limit=(3, 5), p=0.1),
    A.ISONoise(p=0.2),
], bbox_params=A.BboxParams(format='yolo', min_visibility=0.3, label_fields=['class_labels']))

image_paths = glob(f"{INPUT_DIR}/**/*.jpg", recursive=True) + \
              glob(f"{INPUT_DIR}/**/*.png", recursive=True) + \
              glob(f"{INPUT_DIR}/**/*.jpeg", recursive=True)

def convert_to_yolo_bbox(parts):
    try:
        coords = list(map(float, parts))
        
        xs = coords[0::2]
        ys = coords[1::2]
        
        if not xs or not ys: return None
        
        min_x, max_x = min(xs), max(xs)
        min_y, max_y = min(ys), max(ys)
        
        width = max_x - min_x
        height = max_y - min_y
        x_center = min_x + width / 2
        y_center = min_y + height / 2
        
        x_center = max(0, min(1, x_center))
        y_center = max(0, min(1, y_center))
        width = max(0, min(1, width))
        height = max(0, min(1, height))
        
        return [x_center, y_center, width, height]
    except:
        return None

print(f"Знайдено вхідних зображень: {len(image_paths)}")

for img_path in tqdm(image_paths, desc="Processing"):
    try:
        label_path = img_path.replace("/images/", "/labels/").rsplit('.', 1)[0] + ".txt"

        if not os.path.exists(label_path):
            label_path = img_path.rsplit('.', 1)[0] + ".txt"
            
        if not os.path.exists(label_path):
            continue

        image = cv2.imread(img_path)
        if image is None: 
            continue
            
        bboxes = []
        class_labels = []
        
        with open(label_path, 'r') as f:
            lines = f.readlines()

        for line in lines:
            parts = line.strip().split()
            if len(parts) > 1:
                cls = int(float(parts[0]))
                if len(parts) == 5:
                    bbox = list(map(float, parts[1:]))
                else:
                    bbox = convert_to_yolo_bbox(parts[1:])
                
                if bbox:
                    if bbox[2] > 0 and bbox[3] > 0:
                        bboxes.append(bbox)
                        class_labels.append(cls)
        
        if not bboxes:
            continue

        base_name = os.path.splitext(os.path.basename(img_path))[0]
        cv2.imwrite(f"{OUTPUT_DIR}/images/{base_name}.jpg", image)
        with open(f"{OUTPUT_DIR}/labels/{base_name}.txt", 'w') as f:
            for bbox, cls in zip(bboxes, class_labels):
                f.write(f"{cls} {' '.join(map(str, bbox))}\n")

        multiplier = 8
        for i in range(multiplier):
            try:
                augmented = transform(image=image, bboxes=bboxes, class_labels=class_labels)
                
                if len(augmented['bboxes']) == 0: continue
                
                new_name = f"{base_name}_aug_{i}"

                cv2.imwrite(f"{OUTPUT_DIR}/images/{new_name}.jpg", augmented['image'])

                with open(f"{OUTPUT_DIR}/labels/{new_name}.txt", 'w') as f:
                    for bbox, cls in zip(augmented['bboxes'], augmented['class_labels']):
                        #bbox = clip_bbox(bbox)
                        f.write(f"{cls} {' '.join(map(str, bbox))}\n")
                
            except Exception as e:
                pass

    except Exception as e:
        print(f"Error: {os.path.basename(img_path)}: {e}")

print(f"Всього зображень після аугментації: {len(os.listdir(f'{OUTPUT_DIR}/images'))}")

Знайдено вхідних зображень: 176


Processing: 100%|██████████| 176/176 [00:27<00:00,  6.50it/s]

Всього зображень після аугментації: 1569


In [4]:
!pip install hub-sdk
import yaml
from ultralytics import YOLO

data_config = {
    'train': '/kaggle/working/augmented_dataset/images', 
    'val': '/kaggle/input/mkr-logos/mkr nns logos.v2i.yolov11/valid/images',
    'nc': 1,            
    'names': ['logo']   
}

with open('/kaggle/working/data.yaml', 'w') as f:
    yaml.dump(data_config, f)
    
model = YOLO('yolo11m.pt') 

results = model.train(
    data='/kaggle/working/data.yaml',
    epochs=25,
    imgsz=640,
    batch=16,
    name='logo_detector_v11',
    project='/kaggle/working/runs'
)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 kB 1.3 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.234 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=25, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False,

/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all         50         58          1      0.927      0.956      0.854
Speed: 0.2ms preprocess, 12.2ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to /kaggle/working/runs/logo_detector_v11


In [5]:
import cv2

video_path = "/kaggle/input/logotypes/logos/logo.mp4"
output_path = "/kaggle/working/logo_detected.mp4"

best_model_path = "/kaggle/working/runs/logo_detector_v11/weights/best.pt"
model = YOLO(best_model_path)

cap = cv2.VideoCapture(video_path)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

print("Обробка відео...")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
        
    results = model.predict(frame, conf=0.9, verbose=False)
    annotated_frame = results[0].plot()
    out.write(annotated_frame)

cap.release()
out.release()
print(f"Відео збережено: {output_path}")

Обробка відео...
Відео збережено: /kaggle/working/logo_detected.mp4
